# Thapar Summer School - Employee Salary Prediction

In this lab, we are predict the salaries of employee from different fake companies.

Here, the features are explain below:

- `id`: Identity of the employee
- `salary`: (Target Column) Salary of the employee
- `company`: Current Company of the employee
- `department`: Department of the employee
- `age`: Current age of the employee
- `age_when_joined`: Employee's age when joined the company
- `year_in_the_company`: Employee's experience in the company
- `prior_years_experiences`: Employee's experience prior to joining the company
- `annual_bonus`: Annual bonus of employee

## Tools

Now, before we get, let's initialize some important's libraries.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Viewing the data

Taking a look in our training dataset and getting more familiar with it. Remember that the training dataset and test dataset are already separated.

In [3]:
df_train = pd.read_csv('train.csv')

Looking at the first five examples of the dataset.

In [4]:
df_train.head()

,id,company,department,age,age_when_joined,years_in_the_company,salary,annual_bonus,prior_years_experience,full_time,part_time,contractor
0,1,Cheerper,Support,40,36,4,69420.46872,22586.99591,2,0.000000,0.893809,0.328591
1,2,Cheerper,BigData,40,34,6,88407.04974,18676.07837,3,0.205947,0.756632,0.036870
2,3,Pear,Sales,41,39,2,97831.84885,19287.87365,2,0.942309,0.000000,0.514457
3,4,Glasses,Search Engine,39,33,6,93905.86813,17936.39297,3,0.484373,0.236922,0.278535
4,5,Glasses,AI,39,35,3,105983.97520,16854.92943,3,0.835346,0.308958,0.000000


Seeing the shape of our dataset.

In [5]:
n_rows = df_train.shape[0]
n_columns = df_train.shape[1]

print(f"The number of rows is: {n_rows}")
print(f"The number of columns is: {n_columns}")

The number of rows is: 100000
The number of columns is: 12


In [6]:
df_train.dtypes

id                          int64
company                    object
department                 object
age                         int64
age_when_joined             int64
years_in_the_company        int64
salary                    float64
annual_bonus              float64
prior_years_experience      int64
full_time                 float64
part_time                 float64
contractor                float64
dtype: object

Checking if the dataframe has some null value.

In [7]:
df_train.isnull().sum()

id                        0
company                   0
department                0
age                       0
age_when_joined           0
years_in_the_company      0
salary                    0
annual_bonus              0
prior_years_experience    0
full_time                 0
part_time                 0
contractor                0
dtype: int64

That's good. Now, we notice that just two features are categorical (`company` and `department`), while the others are numerical (without considerer `id`). Let's see the different values that each of the two feature contains.

#### Values in _company_ and _department_

In [13]:
df_train['company'].value_counts()

Glasses     47734
Cheerper    28583
Pear        23683
Name: company, dtype: int64

In [14]:
df_train['department'].value_counts()

Search Engine    21915
AI               21642
BigData          15777
Design           15713
Sales            12535
Support          12418
Name: department, dtype: int64

In [17]:
df_train.describe(include=object)

,company,department
count,100000,100000
unique,3,6
top,Glasses,Search Engine
freq,47734,21915


#### Summary measures of each numerical feature

In [18]:
df_train.describe(include=[int, float])

,id,age,age_when_joined,years_in_the_company,salary,annual_bonus,prior_years_experience,full_time,part_time,contractor
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,50000.500000,38.195870,33.537060,4.667780,87389.018245,18582.064815,2.505540,0.383531,0.383279,0.382016
std,28867.657797,6.013073,7.719078,2.531773,28478.542805,4373.431365,1.207222,0.340638,0.339783,0.339621
min,1.000000,30.000000,22.000000,1.000000,40000.000000,9000.000000,1.000000,0.000000,0.000000,0.000000
25%,25000.750000,33.000000,27.000000,3.000000,66027.961360,15448.885482,1.000000,0.016171,0.015702,0.013575
50%,50000.500000,38.000000,33.000000,5.000000,86554.204990,18821.651785,2.000000,0.333278,0.334739,0.331437
75%,75000.250000,43.000000,39.000000,7.000000,107269.920325,22168.522630,3.000000,0.653879,0.650733,0.649338
max,100000.000000,49.000000,48.000000,9.000000,153000.000000,24792.910000,5.000000,1.000000,1.000000,1.000000


For we will start the data pre-processing, we know that categorical features like **company** and **department** have a lower cardinality, which can be managed by some **encoding** techniques. 

Also, we know that the others features (which are numerical) are having **high deviation** values which can lead to bias, overfitting, and can affect the accuracy of the model. So, we can use some different transformtion techniques to reduce the deviation between the data-points.